In [1]:
import pandas as pd

datesDf = pd.read_excel('..\data\T1Dexi\manifestcombo.xlsx', sheet_name="Dates2Analyze")
manifestDf = pd.read_excel('..\data\T1Dexi\manifestcombo.xlsx', sheet_name="Manifests", dtype={"MealID": str})

In [2]:
dayCols = []
for i in range(1, 13):
    dayCols.append(f'Day {i}')
    
subjects = manifestDf['SubjectId'].drop_duplicates()
subjects

0        01-0007
71       01-0010
180      01-0028
195      01-0030
212      01-0036
          ...   
14678    01-0594
14726    01-0240
15333    01-0548
15586    01-0657
16436    01-0514
Name: SubjectId, Length: 200, dtype: object

In [3]:
manifestDf["NormDate"] = manifestDf["Date"].dt.normalize()
manifestDf["NormDate"]

0       2020-02-15
1       2020-02-15
2       2020-02-12
3       2020-02-12
4       2020-02-12
           ...    
16910   2020-09-16
16911   2020-09-16
16912   2020-09-16
16913   2020-09-16
16914   2020-09-16
Name: NormDate, Length: 16915, dtype: datetime64[ns]

In [4]:
normDates = datesDf.set_index('id')[dayCols]
for col in dayCols:
    normDates[col] = normDates[col].dt.normalize()

normDates.head()

,Day 1,Day 2,Day 3,Day 4,Day 5,Day 6,Day 7,Day 8,Day 9,Day 10,Day 11,Day 12
id,,,,,,,,,,,,
01-0007,2020-02-03,2020-02-04,2020-02-07,2020-02-08,2020-02-11,2020-02-12,NaT,NaT,NaT,NaT,NaT,NaT
01-0010,2020-02-07,2020-02-08,2020-02-11,2020-02-12,2020-02-14,2020-02-15,2020-02-18,2020-02-19,2020-02-21,2020-02-22,2020-02-28,2020-02-29
01-0023,2020-07-17,2020-07-18,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
01-0028,2020-02-13,2020-02-14,2020-02-18,2020-02-19,2020-02-20,2020-02-21,2020-02-25,2020-02-26,2020-03-02,2020-03-03,2020-03-09,2020-03-10
01-0029,2020-02-24,2020-02-25,2020-02-27,2020-02-28,2020-03-02,2020-03-03,2020-03-05,2020-03-06,2020-03-09,2020-03-10,2020-03-17,2020-03-18


In [5]:
onlyReleventDatesDf = pd.DataFrame()
for subject in subjects:
    try:
        subFilt = manifestDf['SubjectId'] == subject
        subFiltDf = manifestDf[subFilt]
        normFilt = subFiltDf['NormDate'].isin(normDates.loc[subject].dropna())
        onlyReleventDatesDf = onlyReleventDatesDf.append(subFiltDf[normFilt])
    except KeyError:
        print(subject)

00-0000
01-0514


In [6]:
onlyReleventDatesDf.head()

,ImageId,SubjectId,Date,Time,Pre/Post Photo,MealID,MealTypeDesc,MealDescription,NormDate
2,10100071581552343255_01-0007_2020-02-12_19-06-...,01-0007,2020-02-12,19:06 EST,Pre Meal,10100071581552343255,Dinner,"little neck clams, sauteed broccoli raabe, roa...",2020-02-12
3,10100071581552343255_01-0007_2020-02-12_19-26-...,01-0007,2020-02-12,19:26 EST,Post Meal,10100071581552343255,Dinner,"little neck clams, sauteed broccoli raabe, roa...",2020-02-12
4,10100071581532019144_01-0007_2020-02-12_13-31-...,01-0007,2020-02-12,13:31 EST,Pre Meal,10100071581532019144,Lunch,NaN,2020-02-12
5,10100071581532019144_01-0007_2020-02-12_14-08-...,01-0007,2020-02-12,14:08 EST,Post Meal,10100071581532019144,Lunch,NaN,2020-02-12
6,10100071581524042751_01-0007_2020-02-12_11-14-...,01-0007,2020-02-12,11:14 EST,Pre Meal,10100071581524042751,Morning Snack,chomps turkey stick,2020-02-12


<h2>Checking out the errors</h2>

In [7]:
manifestDf[manifestDf['SubjectId'] == '01-0514']

,ImageId,SubjectId,Date,Time,Pre/Post Photo,MealID,MealTypeDesc,MealDescription,NormDate
16436,10105141600706676021_01-0514_2020-09-21_12-44-...,01-0514,2020-09-21,12:44 Eastern Daylight Time,Pre Meal,10105141600706676021,Lunch,"Glass of 2% milk, gluten free penne pasta with...",2020-09-21
16437,10105141600706676021_01-0514_2020-09-21_13-24-...,01-0514,2020-09-21,13:24 Eastern Daylight Time,Post Meal,10105141600706676021,Lunch,"Glass of 2% milk, gluten free penne pasta with...",2020-09-21
16459,10105141600696264699_01-0514_2020-09-21_09-51-...,01-0514,2020-09-21,09:51 Eastern Daylight Time,Pre Meal,10105141600696264699,Breakfast,Glass of 2% milk,2020-09-21
16460,10105141600696264699_01-0514_2020-09-21_10-14-...,01-0514,2020-09-21,10:14 Eastern Daylight Time,Post Meal,10105141600696264699,Breakfast,Glass of 2% milk,2020-09-21


In [8]:
#normDates.loc['01-0514']
#this is the error

In [9]:
onlyReleventDatesDf.head()

,ImageId,SubjectId,Date,Time,Pre/Post Photo,MealID,MealTypeDesc,MealDescription,NormDate
2,10100071581552343255_01-0007_2020-02-12_19-06-...,01-0007,2020-02-12,19:06 EST,Pre Meal,10100071581552343255,Dinner,"little neck clams, sauteed broccoli raabe, roa...",2020-02-12
3,10100071581552343255_01-0007_2020-02-12_19-26-...,01-0007,2020-02-12,19:26 EST,Post Meal,10100071581552343255,Dinner,"little neck clams, sauteed broccoli raabe, roa...",2020-02-12
4,10100071581532019144_01-0007_2020-02-12_13-31-...,01-0007,2020-02-12,13:31 EST,Pre Meal,10100071581532019144,Lunch,NaN,2020-02-12
5,10100071581532019144_01-0007_2020-02-12_14-08-...,01-0007,2020-02-12,14:08 EST,Post Meal,10100071581532019144,Lunch,NaN,2020-02-12
6,10100071581524042751_01-0007_2020-02-12_11-14-...,01-0007,2020-02-12,11:14 EST,Pre Meal,10100071581524042751,Morning Snack,chomps turkey stick,2020-02-12


In [10]:
noRowDupDf = onlyReleventDatesDf.drop_duplicates()
noRowDupDf.head()

,ImageId,SubjectId,Date,Time,Pre/Post Photo,MealID,MealTypeDesc,MealDescription,NormDate
2,10100071581552343255_01-0007_2020-02-12_19-06-...,01-0007,2020-02-12,19:06 EST,Pre Meal,10100071581552343255,Dinner,"little neck clams, sauteed broccoli raabe, roa...",2020-02-12
3,10100071581552343255_01-0007_2020-02-12_19-26-...,01-0007,2020-02-12,19:26 EST,Post Meal,10100071581552343255,Dinner,"little neck clams, sauteed broccoli raabe, roa...",2020-02-12
4,10100071581532019144_01-0007_2020-02-12_13-31-...,01-0007,2020-02-12,13:31 EST,Pre Meal,10100071581532019144,Lunch,NaN,2020-02-12
5,10100071581532019144_01-0007_2020-02-12_14-08-...,01-0007,2020-02-12,14:08 EST,Post Meal,10100071581532019144,Lunch,NaN,2020-02-12
6,10100071581524042751_01-0007_2020-02-12_11-14-...,01-0007,2020-02-12,11:14 EST,Pre Meal,10100071581524042751,Morning Snack,chomps turkey stick,2020-02-12


In [11]:
preMealFilter = noRowDupDf['Pre/Post Photo'] == "Pre Meal"
postMealFilter = noRowDupDf['Pre/Post Photo'] == "Post Meal"
preDf = noRowDupDf[preMealFilter]
postDf = noRowDupDf[postMealFilter]

<h2>Empty Dataframe shows that there are no duplicated meal ids</h2>

In [12]:
postDf[postDf['MealID'].duplicated()]

,ImageId,SubjectId,Date,Time,Pre/Post Photo,MealID,MealTypeDesc,MealDescription,NormDate


In [13]:
postDf = postDf.set_index('MealID')

In [14]:
postMealIDs = postDf.index
postDf.head()

,ImageId,SubjectId,Date,Time,Pre/Post Photo,MealTypeDesc,MealDescription,NormDate
MealID,,,,,,,,
10100071581552343255,10100071581552343255_01-0007_2020-02-12_19-26-...,01-0007,2020-02-12,19:26 EST,Post Meal,Dinner,"little neck clams, sauteed broccoli raabe, roa...",2020-02-12
10100071581532019144,10100071581532019144_01-0007_2020-02-12_14-08-...,01-0007,2020-02-12,14:08 EST,Post Meal,Lunch,NaN,2020-02-12
10100071581524042751,10100071581524042751_01-0007_2020-02-12_11-35-...,01-0007,2020-02-12,11:35 EST,Post Meal,Morning Snack,chomps turkey stick,2020-02-12
10100071581462720084,10100071581462720084_01-0007_2020-02-11_18-24-...,01-0007,2020-02-11,18:24 EST,Post Meal,Dinner,"dogfish head slightly mighty local ipa, 2 chic...",2020-02-11
10100071581439850586,10100071581439850586_01-0007_2020-02-11_13-58-...,01-0007,2020-02-11,13:58 EST,Post Meal,Lunch,kim chee amd chicken breast bites,2020-02-11


<h2>Only 10100291583544047632 is duplicated</h2>

In [15]:
preDf[preDf['MealID'].duplicated()]

,ImageId,SubjectId,Date,Time,Pre/Post Photo,MealID,MealTypeDesc,MealDescription,NormDate
3484,10100291583544047632_01-0029_2020-03-06_20-21-...,01-0029,2020-03-06,20:21 EST,Pre Meal,10100291583544047632,Afternoon Snack,big handful pretzels with 1/2 cup cream cheese...,2020-03-06


In [16]:
preDf[preDf['MealID']=='10100291583544047632']

,ImageId,SubjectId,Date,Time,Pre/Post Photo,MealID,MealTypeDesc,MealDescription,NormDate
2898,10100291583544047632_01-0029_2020-03-06_20-21-...,01-0029,2020-03-06,20:21 EST,Pre Meal,10100291583544047632,Afternoon Snack,.,2020-03-06
3484,10100291583544047632_01-0029_2020-03-06_20-21-...,01-0029,2020-03-06,20:21 EST,Pre Meal,10100291583544047632,Afternoon Snack,big handful pretzels with 1/2 cup cream cheese...,2020-03-06


In [17]:
preDf = preDf.drop(index=[2898])

In [18]:
preDf[preDf['MealID'].duplicated()]

,ImageId,SubjectId,Date,Time,Pre/Post Photo,MealID,MealTypeDesc,MealDescription,NormDate


<h2>Adding the post meal information to the pre meal information.</h2>

<p>There are more before photos than after photos. So if they're not in the list of post meal ids then the meal has no post meal info</p>

In [19]:
preDf.loc[:,'PostMealImageID']='No Post Image'
preDf.loc[:,'PostMealImageTime']='No Post Image'

In [20]:
preDf.head()

,ImageId,SubjectId,Date,Time,Pre/Post Photo,MealID,MealTypeDesc,MealDescription,NormDate,PostMealImageID,PostMealImageTime
2,10100071581552343255_01-0007_2020-02-12_19-06-...,01-0007,2020-02-12,19:06 EST,Pre Meal,10100071581552343255,Dinner,"little neck clams, sauteed broccoli raabe, roa...",2020-02-12,No Post Image,No Post Image
4,10100071581532019144_01-0007_2020-02-12_13-31-...,01-0007,2020-02-12,13:31 EST,Pre Meal,10100071581532019144,Lunch,NaN,2020-02-12,No Post Image,No Post Image
6,10100071581524042751_01-0007_2020-02-12_11-14-...,01-0007,2020-02-12,11:14 EST,Pre Meal,10100071581524042751,Morning Snack,chomps turkey stick,2020-02-12,No Post Image,No Post Image
8,10100071581512374015_01-0007_2020-02-12_08-00-...,01-0007,2020-02-12,08:00 EST,Pre Meal,10100071581512374015,Breakfast,"grande cofffee, kind bar",2020-02-12,No Post Image,No Post Image
9,10100071581462720084_01-0007_2020-02-11_18-13-...,01-0007,2020-02-11,18:13 EST,Pre Meal,10100071581462720084,Dinner,"dogfish head slightly mighty local ipa, 2 chic...",2020-02-11,No Post Image,No Post Image


In [21]:
for id in postMealIDs:
    postMealImgID = postDf.loc[id]['ImageId']
    postMealImageTime = postDf.loc[id]['Time']
    meal = preDf[preDf['MealID']==id]
    preDf.loc[meal.index, 'PostMealImageID'] = postMealImgID
    preDf.loc[meal.index, 'PostMealImageTime'] = postMealImageTime


In [22]:
preDf.head()

,ImageId,SubjectId,Date,Time,Pre/Post Photo,MealID,MealTypeDesc,MealDescription,NormDate,PostMealImageID,PostMealImageTime
2,10100071581552343255_01-0007_2020-02-12_19-06-...,01-0007,2020-02-12,19:06 EST,Pre Meal,10100071581552343255,Dinner,"little neck clams, sauteed broccoli raabe, roa...",2020-02-12,10100071581552343255_01-0007_2020-02-12_19-26-...,19:26 EST
4,10100071581532019144_01-0007_2020-02-12_13-31-...,01-0007,2020-02-12,13:31 EST,Pre Meal,10100071581532019144,Lunch,NaN,2020-02-12,10100071581532019144_01-0007_2020-02-12_14-08-...,14:08 EST
6,10100071581524042751_01-0007_2020-02-12_11-14-...,01-0007,2020-02-12,11:14 EST,Pre Meal,10100071581524042751,Morning Snack,chomps turkey stick,2020-02-12,10100071581524042751_01-0007_2020-02-12_11-35-...,11:35 EST
8,10100071581512374015_01-0007_2020-02-12_08-00-...,01-0007,2020-02-12,08:00 EST,Pre Meal,10100071581512374015,Breakfast,"grande cofffee, kind bar",2020-02-12,No Post Image,No Post Image
9,10100071581462720084_01-0007_2020-02-11_18-13-...,01-0007,2020-02-11,18:13 EST,Pre Meal,10100071581462720084,Dinner,"dogfish head slightly mighty local ipa, 2 chic...",2020-02-11,10100071581462720084_01-0007_2020-02-11_18-24-...,18:24 EST


In [23]:
preDf[preDf['MealID'].duplicated()]

,ImageId,SubjectId,Date,Time,Pre/Post Photo,MealID,MealTypeDesc,MealDescription,NormDate,PostMealImageID,PostMealImageTime


In [24]:
preDf = preDf.set_index('MealID')
preDf.rename(mapper={
    'Time':'PreMealImageTime',
    "ImageId":"PreMealImageID",
    "MealTypeDesc": "Meal",
    "MealDescription": "Description"}, inplace=True, axis=1)

desiredCols = ['SubjectId',
               'Date',
               'Meal',
               'Description',
               'PreMealImageTime',
               'PostMealImageTime',
               "PreMealImageID",
               "PostMealImageID"]

In [25]:
#preDf[desiredCols].to_excel('tested.xlsx')
#commented out b/c made some cosmetic changes